In [22]:
import pandas as pd
import numpy as np
import requests

In [23]:
canada_data_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
candada_data_page = requests.get(canada_data_url)

candada_data = pd.read_html(candada_data_page.content, header = 0)[0]

In [24]:
candada_data
candada_data = candada_data[candada_data.Neighbourhood != 'Not assigned']
candada_data.reset_index(inplace = True,drop=True)
candada_data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
candada_data.groupby(['Postal Code']).first()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [26]:
len(candada_data['Postal Code'].unique())

103

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough by the below code.

In [27]:
candada_data["Neighbourhood"].replace({"Not assigned": candada_data["Borough"]}, inplace=True)

In [28]:
candada_data.shape

(103, 3)

# Part 2

In [31]:
import pandas as pd

In [30]:
import geocoder # import geocoder
def get_latitude_longitude(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [41]:
toronto_neighbourhood=candada_data
toronto_neighbourhood=toronto_neighbourhood[["Postal Code","Borough","Neighbourhood"]]

Below code will add Latitude and Longitude columns in our toronto dataset

```python
toronto_neighbourhood.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_neighbourhood["Latitude"],toronto_neighbourhood["Longitude"]=toronto_neighbourhood.apply(lambda row: get_latitude_longitude(row.PostalCode),axis=1)

```

---
**NOTE**

Geocoder was not working for me hence used the csv provided.

---

In [40]:
url='http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's check both dataframes have same number of rows or not

In [42]:
toronto_neighbourhood.shape==df_geo.shape

True

In [43]:
toronto_neighbourhood = toronto_neighbourhood.join(df_geo.set_index('Postal Code'), on='Postal Code')
toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [35]:
toronto_neighbourhood.reset_index(drop=True,inplace=True)

In [36]:
toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
